<!-- ---
title: Distributed Training on CPUs, GPUs or TPUs
weight: 1
date: 2021-09-18
downloads: true
tags:
  - multi GPUs on a single node
  - multi GPUs on multiple nodes
  - TPUs on Colab
  - Jupyter Notebooks
--- -->
# Distributed Training with Ignite on CIFAR10 

This tutorial is a brief introduction on how you can do distributed training with Ignite on one or more CPUs, GPUs or TPUs. We will also introduce several helper functions and Ignite concepts (setup common training handlers, save to/ load from checkpoints, etc.) which you can easily incorporate in your code.

<!--more-->

We will use distributed training to train a predefined [ResNet18](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet18) on [CIFAR10](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.CIFAR10) using either of the following configurations:

* Single Node, One or More GPUs
* Multiple Nodes, Multiple GPUs
* Single Node, Multiple CPUs
* TPUs on Google Colab
* On Jupyter Notebooks

The type of distributed training we will use is called data parallelism in which we:

>   1. Copy the model on each GPU
>   2. Split the dataset and fit the models on different subsets
>   3. Communicate the gradients at each iteration to keep the models in sync
>
> -- <cite>[Distributed Deep Learning 101: Introduction](https://towardsdatascience.com/distributed-deep-learning-101-introduction-ebfc1bcd59d9)</cite>

PyTorch provides a [torch.nn.parallel.DistributedDataParallel](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html) API for this task however the implementation that supports different backends + configurations is tedious. In this example, we will see how to can enable data distributed training which is adaptable to various backends in just a few lines of code alongwith:
* Computing training and validation metrics
* Setup logging (and connecting with ClearML)
* Saving the best model weights
* Setting LR Scheduler
* Using Automatic Mixed Precision

## Required Dependencies

In [ ]:
!pip install pytorch-ignite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 3.2 MB/s eta 0:00:00


### For parsing arguments

In [ ]:
!pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=7ed6a87a1bb57d5431796665db8b2b2738721af0f12ae4cf5087eda9d4b426ec
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire


## Download Data

Let's download our data first which can later be used by all the processes to instantiate our dataloaders. The following command will download the CIFAR10 dataset to a folder `cifar10`.

In [ ]:
!python -c "from torchvision.datasets import CIFAR10; CIFAR10('cifar10', download=True)"

100% 170498071/170498071 [00:01<00:00, 89775284.73it/s]
Extracting cifar10/cifar-10-python.tar.gz to cifar10


## Common Configuration

We maintain a `config` dictionary which can be extended or changed to store parameters required during training. We can refer back to this code when we will use these parameters later.

In [ ]:
config = {
    "seed": 543,
    "data_path": "cifar10",
    "output_path": "output-cifar10/",
    "model": "resnet18",
    "batch_size": 512,
    "momentum": 0.9,
    "weight_decay": 1e-4,
    "num_workers": 2,
    "num_epochs": 5,
    "learning_rate": 0.4,
    "num_warmup_epochs": 1,
    "validate_every": 3,
    "checkpoint_every": 200,
    "backend": None,
    "resume_from": None,
    "log_every_iters": 15,
    "nproc_per_node": None,
    "with_clearml": False,
    "with_amp": False,
}

## Basic Setup

### Imports

In [ ]:
from datetime import datetime
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torchvision.transforms import (
    Compose,
    Normalize,
    Pad,
    RandomCrop,
    RandomHorizontalFlip,
    ToTensor,
)

import ignite
import ignite.distributed as idist
from ignite.contrib.engines import common
from ignite.handlers import PiecewiseLinear
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.handlers import Checkpoint, global_step_from_engine
from ignite.metrics import Accuracy, Loss
from ignite.utils import manual_seed, setup_logger

Next we will take the help of `auto_` methods in `idist` ([`ignite.distributed`](https://pytorch.org/ignite/distributed.html#)) to make our dataloaders, model and optimizer automatically adapt to the current configuration `backend=None` (non-distributed) or for backends like `nccl`, `gloo`, and `xla-tpu` (distributed).

Note that we are free to partially use or not use `auto_` methods at all and instead can implement something custom.

### Dataloaders

Next we are going to instantiate the train and test datasets from `data_path`, apply transforms to it and return them via `get_train_test_datasets()`.

In [ ]:
def get_train_test_datasets(path):
    train_transform = Compose(
        [
            Pad(4),
            RandomCrop(32, fill=128),
            RandomHorizontalFlip(),
            ToTensor(),
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )
    test_transform = Compose(
        [
            ToTensor(),
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

    train_ds = datasets.CIFAR10(
        root=path, train=True, download=False, transform=train_transform
    )
    test_ds = datasets.CIFAR10(
        root=path, train=False, download=False, transform=test_transform
    )

    return train_ds, test_ds

Finally, we pass the datasets to [`auto_dataloader()`](https://pytorch.org/ignite/generated/ignite.distributed.auto.auto_dataloader.html#ignite.distributed.auto.auto_dataloader).

In [ ]:
def get_dataflow(config):
    train_dataset, test_dataset = get_train_test_datasets(config["data_path"])

    train_loader = idist.auto_dataloader(
        train_dataset,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        shuffle=True,
        drop_last=True,
    )

    test_loader = idist.auto_dataloader(
        test_dataset,
        batch_size=2 * config["batch_size"],
        num_workers=config["num_workers"],
        shuffle=False,
    )
    return train_loader, test_loader

### Model

We check if the model given in `config` is present in [torchvision.models](https://pytorch.org/vision/stable/models.html), change the last layer to output 10 classes (as present in CIFAR10) and pass it to [`auto_model()`](https://pytorch.org/ignite/generated/ignite.distributed.auto.auto_model.html#auto-model) which makes it automatically adaptable for non-distributed and distributed configurations.


In [ ]:
def get_model(config):
    model_name = config["model"]
    if model_name in models.__dict__:
        fn = models.__dict__[model_name]
    else:
        raise RuntimeError(f"Unknown model name {model_name}")

    model = idist.auto_model(fn(num_classes=10))

    return model

### Optimizer

Then we can setup the optimizer using hyperameters from `config` and pass it through [`auto_optim()`](https://pytorch.org/ignite/generated/ignite.distributed.auto.auto_optim.html#ignite.distributed.auto.auto_optim).

In [ ]:
def get_optimizer(config, model):
    optimizer = optim.SGD(
        model.parameters(),
        lr=config["learning_rate"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
        nesterov=True,
    )
    optimizer = idist.auto_optim(optimizer)

    return optimizer

### Criterion

We put the loss function on `device`.

In [ ]:
def get_criterion():
    return nn.CrossEntropyLoss().to(idist.device())

### LR Scheduler

We will use [PiecewiseLinear](https://pytorch.org/ignite/generated/ignite.handlers.param_scheduler.PiecewiseLinear.html#ignite.handlers.param_scheduler.PiecewiseLinear) which is one of the [various LR Schedulers](https://pytorch.org/ignite/handlers.html#parameter-scheduler) Ignite provides.


In [ ]:
def get_lr_scheduler(config, optimizer):
    milestones_values = [
        (0, 0.0),
        (config["num_iters_per_epoch"] * config["num_warmup_epochs"], config["learning_rate"]),
        (config["num_iters_per_epoch"] * config["num_epochs"], 0.0),
    ]
    lr_scheduler = PiecewiseLinear(
        optimizer, param_name="lr", milestones_values=milestones_values
    )
    return lr_scheduler

## Trainer

### Save Models

We can create checkpoints using either a handler (in case of ClearML) or by simply passing the path of the checkpoint file to `save_handler`:
If specified `with-clearml=True`, we will save the models in ClearML's File Server using [`ClearMLSaver()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.clearml_logger.html#ignite.contrib.handlers.clearml_logger.ClearMLSaver).

In [ ]:
def get_save_handler(config):
    if config["with_clearml"]:
        from ignite.contrib.handlers.clearml_logger import ClearMLSaver

        return ClearMLSaver(dirname=config["output_path"])

    return config["output_path"]

### Resume from Checkpoint

If a checkpoint file path is provided, we can resume training from there by loading the file.

In [ ]:
def load_checkpoint(resume_from):
    checkpoint_fp = Path(resume_from)
    assert (
        checkpoint_fp.exists()
    ), f"Checkpoint '{checkpoint_fp.as_posix()}' is not found"
    checkpoint = torch.load(checkpoint_fp.as_posix(), map_location="cpu")
    return checkpoint

### Create Trainer

Finally, we can create our `trainer` in four steps:
1. Create a `trainer` object using [`create_supervised_trainer()`](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_trainer.html#ignite.engine.create_supervised_trainer) which internally defines the steps taken to process a single batch:
  1. Move the batch to `device` used in current distributed configuration.
  2. Put `model` in `train()` mode.
  3. Perform forward pass by passing the inputs through the `model` and calculating `loss`. If AMP is enabled then this step happens with [`autocast`](https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.autocast) on which allows this step to run in mixed precision.
  4. Perform backward pass. If [Automatic Mixed Precision](https://pytorch.org/docs/stable/amp.html) (AMP) is enabled (speeds up computations on large neural networks and reduces memory usage while retaining performance), then the losses will be [scaled](https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler.scale) before calling `backward()`, `step()` the optimizer while discarding batches that contain NaNs and [update()](https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler.update) the scale for the next iteration.
  5. Store the loss as `batch loss` in `state.output`.

Internally, the above steps to create the `trainer` would look like:
  ```python
  def train_step(engine, batch):

        x, y = batch[0], batch[1]
        if x.device != device:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

        model.train()

        with autocast(enabled=with_amp):
            y_pred = model(x)
            loss = criterion(y_pred, y)

        optimizer.zero_grad()
        scaler.scale(loss).backward()  # If with_amp=False, this is equivalent to loss.backward()
        scaler.step(optimizer)  # If with_amp=False, this is equivalent to optimizer.step()
        scaler.update()  # If with_amp=False, this step does nothing

        return {"batch loss": loss.item()}

  trainer = Engine(train_step)
  ```
3. Setup some common Ignite training handlers. You can do this individually or use [setup_common_training_handlers()](https://pytorch.org/ignite/contrib/engines.html#ignite.contrib.engines.common.setup_common_training_handlers) that takes the `trainer` and a subset of the dataset (`train_sampler`) alongwith:
  * A dictionary mapping on what to save in the checkpoint (`to_save`) and how often (`save_every_iters`).
  * The LR Scheduler
  * The output of `train_step()`
  * Other handlers
4. If `resume_from` file path is provided, load the states of objects `to_save` from the checkpoint file.

In [ ]:
def create_trainer(
    model, optimizer, criterion, lr_scheduler, train_sampler, config, logger
):

    device = idist.device()
    amp_mode = None
    scaler = False
        
    trainer = create_supervised_trainer(
        model,
        optimizer,
        criterion,
        device=device,
        non_blocking=True,
        output_transform=lambda x, y, y_pred, loss: {"batch loss": loss.item()},
        amp_mode="amp" if config["with_amp"] else None,
        scaler=config["with_amp"],
    )
    trainer.logger = logger

    to_save = {
        "trainer": trainer,
        "model": model,
        "optimizer": optimizer,
        "lr_scheduler": lr_scheduler,
    }
    metric_names = [
        "batch loss",
    ]

    common.setup_common_training_handlers(
        trainer=trainer,
        train_sampler=train_sampler,
        to_save=to_save,
        save_every_iters=config["checkpoint_every"],
        save_handler=get_save_handler(config),
        lr_scheduler=lr_scheduler,
        output_names=metric_names if config["log_every_iters"] > 0 else None,
        with_pbars=False,
        clear_cuda_cache=False,
    )

    if config["resume_from"] is not None:
        checkpoint = load_checkpoint(config["resume_from"])
        Checkpoint.load_objects(to_load=to_save, checkpoint=checkpoint)

    return trainer

## Evaluator

The evaluator will be created via [`create_supervised_evaluator()`](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_evaluator.html#ignite.engine.create_supervised_evaluator) which internally will:
1. Set the `model` to `eval()` mode.
2. Move the batch to `device` used in current distributed configuration.
3. Perform forward pass. If AMP is enabled, `autocast` will be on.
4. Store the predictions and labels in `state.output` to compute metrics.

It will also attach the Ignite metrics passed to the `evaluator`. 

In [ ]:
def create_evaluator(model, metrics, config):
    device = idist.device()

    amp_mode = "amp" if config["with_amp"] else None
    evaluator = create_supervised_evaluator(
        model, metrics=metrics, device=device, non_blocking=True, amp_mode=amp_mode
    )
    
    return evaluator

## Training

Before we begin training, we must setup a few things on the master process (`rank` = 0):
* Create folder to store checkpoints, best models and output of tensorboard logging in the format - model_backend_rank_time.
* If ClearML FileServer is used to save models, then a `Task` has to be created, and we pass our `config` dictionary and the specific hyper parameters that are part of the experiment.

In [ ]:
def setup_rank_zero(logger, config):
    device = idist.device()

    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    output_path = config["output_path"]
    folder_name = (
        f"{config['model']}_backend-{idist.backend()}-{idist.get_world_size()}_{now}"
    )
    output_path = Path(output_path) / folder_name
    if not output_path.exists():
        output_path.mkdir(parents=True)
    config["output_path"] = output_path.as_posix()
    logger.info(f"Output path: {config['output_path']}")

    if config["with_clearml"]:
        from clearml import Task

        task = Task.init("CIFAR10-Training", task_name=output_path.stem)
        task.connect_configuration(config)
        # Log hyper parameters
        hyper_params = [
            "model",
            "batch_size",
            "momentum",
            "weight_decay",
            "num_epochs",
            "learning_rate",
            "num_warmup_epochs",
        ]
        task.connect({k: v for k, v in config.items()})

### Logging

This step is optional, however, we can pass a [`setup_logger()`](https://pytorch.org/ignite/utils.html#ignite.utils.setup_logger) object to `log_basic_info()` and log all basic information such as different versions, current configuration, `device` and `backend` used by the current process (identified by its local rank), and number of processes (world size). `idist` (`ignite.distributed`) provides several utility functions like [`get_local_rank()`](https://pytorch.org/ignite/distributed.html#ignite.distributed.utils.get_local_rank), [`backend()`](https://pytorch.org/ignite/distributed.html#ignite.distributed.utils.backend), [`get_world_size()`](https://pytorch.org/ignite/distributed.html#ignite.distributed.utils.get_world_size), etc. to make this possible.


In [ ]:
def log_basic_info(logger, config):
    logger.info(f"Train on CIFAR10")
    logger.info(f"- PyTorch version: {torch.__version__}")
    logger.info(f"- Ignite version: {ignite.__version__}")
    if torch.cuda.is_available():
        # explicitly import cudnn as torch.backends.cudnn can not be pickled with hvd spawning procs
        from torch.backends import cudnn

        logger.info(
            f"- GPU Device: {torch.cuda.get_device_name(idist.get_local_rank())}"
        )
        logger.info(f"- CUDA version: {torch.version.cuda}")
        logger.info(f"- CUDNN version: {cudnn.version()}")

    logger.info("\n")
    logger.info("Configuration:")
    for key, value in config.items():
        logger.info(f"\t{key}: {value}")
    logger.info("\n")

    if idist.get_world_size() > 1:
        logger.info("\nDistributed setting:")
        logger.info(f"\tbackend: {idist.backend()}")
        logger.info(f"\tworld size: {idist.get_world_size()}")
        logger.info("\n")

This is a standard utility function to log `train` and `val` metrics after `validate_every` epochs.

In [ ]:
def log_metrics(logger, epoch, elapsed, tag, metrics):
    metrics_output = "\n".join([f"\t{k}: {v}" for k, v in metrics.items()])
    logger.info(
        f"\nEpoch {epoch} - Evaluation time (seconds): {elapsed:.2f} - {tag} metrics:\n {metrics_output}"
    )

### Begin Training

This is where the main logic resides, i.e. we will call all the above functions from within here:
1. Basic Setup
  1. We set a [`manual_seed()`](https://pytorch.org/ignite/utils.html#ignite.utils.manual_seed) and [`setup_logger()`](https://pytorch.org/ignite/utils.html#ignite.utils.setup_logger), then log all basic information.
  2. Initialise `dataloaders`, `model`, `optimizer`, `criterion` and `lr_scheduler`.
2. We use the above objects to create a `trainer`.
3. Evaluator
  1. Define some relevant Ignite metrics like [`Accuracy()`](https://pytorch.org/ignite/generated/ignite.metrics.Accuracy.html#accuracy) and [`Loss()`](https://pytorch.org/ignite/generated/ignite.metrics.Loss.html#loss).
  2. Create two evaluators: `train_evaluator` and `val_evaluator` to compute metrics on the `train_dataloader` and `val_dataloader` respectively, however `val_evaluator` will store the best models based on validation metrics.
  3. Define `run_validation()` to compute metrics on both dataloaders and log them. Then we attach this function to `trainer` to run after `validate_every` epochs and after training is complete.
4. Setup TensorBoard logging using [`setup_tb_logging()`](https://pytorch.org/ignite/contrib/engines.html#ignite.contrib.engines.common.setup_tb_logging) on the master process for the trainer and evaluators so that training and validation metrics along with the learning rate can be logged.
5. Define a [`Checkpoint()`](https://pytorch.org/ignite/generated/ignite.handlers.checkpoint.Checkpoint.html#ignite.handlers.checkpoint.Checkpoint) object to store the two best models (`n_saved`) by validation accuracy (defined in `metrics` as `Accuracy()`) and attach it to `val_evaluator` so that it can be executed everytime `val_evaluator` runs.
6. Try training on `train_loader` for `num_epochs`
7. Close Tensorboard logger once training is completed.



In [ ]:
def training(local_rank, config):

    rank = idist.get_rank()
    manual_seed(config["seed"] + rank)

    logger = setup_logger(name="CIFAR10-Training")
    log_basic_info(logger, config)

    if rank == 0:
        setup_rank_zero(logger, config)

    train_loader, val_loader = get_dataflow(config)
    model = get_model(config)
    optimizer = get_optimizer(config, model)
    criterion = get_criterion()
    config["num_iters_per_epoch"] = len(train_loader)
    lr_scheduler = get_lr_scheduler(config, optimizer)

    trainer = create_trainer(
        model, optimizer, criterion, lr_scheduler, train_loader.sampler, config, logger
    )

    metrics = {
        "Accuracy": Accuracy(),
        "Loss": Loss(criterion),
    }

    train_evaluator = create_evaluator(model, metrics, config)
    val_evaluator = create_evaluator(model, metrics, config)

    def run_validation(engine):
        epoch = trainer.state.epoch
        state = train_evaluator.run(train_loader)
        log_metrics(logger, epoch, state.times["COMPLETED"], "train", state.metrics)
        state = val_evaluator.run(val_loader)
        log_metrics(logger, epoch, state.times["COMPLETED"], "val", state.metrics)

    trainer.add_event_handler(
        Events.EPOCH_COMPLETED(every=config["validate_every"]) | Events.COMPLETED,
        run_validation,
    )

    if rank == 0:
        evaluators = {"train": train_evaluator, "val": val_evaluator}
        tb_logger = common.setup_tb_logging(
            config["output_path"], trainer, optimizer, evaluators=evaluators
        )

    best_model_handler = Checkpoint(
        {"model": model},
        get_save_handler(config),
        filename_prefix="best",
        n_saved=2,
        global_step_transform=global_step_from_engine(trainer),
        score_name="val_accuracy",
        score_function=Checkpoint.get_default_score_fn("Accuracy"),
    )
    val_evaluator.add_event_handler(
        Events.COMPLETED,
        best_model_handler,
    )

    try:
        trainer.run(train_loader, max_epochs=config["num_epochs"])
    except Exception as e:
        logger.exception("")
        raise e

    if rank == 0:
        tb_logger.close()

## Running Distributed Code

We can easily run the above code with the context manager [Parallel](https://pytorch.org/ignite/generated/ignite.distributed.launcher.Parallel.html#ignite.distributed.launcher.Parallel):

```python
with idist.Parallel(backend=backend, nproc_per_node=nproc_per_node) as parallel:
    parallel.run(training, config)
```
`Parallel` enables us to run the same code across all supported distributed backends and non-distributed configurations in a seamless manner. Here backend refers to a distributed communication framework. Read more about which backend to choose [here](https://pytorch.org/docs/stable/distributed.html#backends). `Parallel` accepts a `backend` and either:

> Spawns `nproc_per_node` child processes and initialize a processing group according to provided backend (useful for standalone scripts).

This way uses `torch.multiprocessing.spawn` and is the default way to spawn processes. However, this way is slower due to initialization overhead. 

or
> Only initialize a processing group given the backend (useful with tools like `torchrun`, `horovodrun`, etc).

This way is recommended since training is faster and easier to extend to multiple scripts.

We can pass additional information to `Parallel` collectively as `spawn_kwargs` as we will see below.

**Note:** It is recommended to run distributed code as scripts for ease of use, however we can also spawn processes in a Jupyter notebook (see end of tutorial). The complete code as a script can be found [here](https://github.com/pytorch-ignite/examples/blob/main/tutorials/intermediate/cifar10-distributed.py). Choose one of the suggested ways below to run the script.

## Run in Jupyter Notebook

We will have to spawn processes in a notebook and therefore, we will use internal spawning to achieve that. For multiple GPUs, use `backend=nccl` and `backend=gloo` for multiple CPUs.

In [ ]:
spawn_kwargs = {}
spawn_kwargs["start_method"] = "fork"
spawn_kwargs["nproc_per_node"] = 1
config["backend"] = "nccl"

with idist.Parallel(backend=config["backend"], **spawn_kwargs) as parallel:
    parallel.run(training, config)

2023-04-15 05:11:34,281 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'nccl'
2023-04-15 05:11:34,283 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 1
	nnodes: 1
	node_rank: 0
	start_method: fork
2023-04-15 05:11:34,285 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f4c18b3bee0>' in 1 processes
2023-04-15 05:11:39,918 CIFAR10-Training INFO: Train on CIFAR10
2023-04-15 05:11:39,920 CIFAR10-Training INFO: - PyTorch version: 2.0.0+cu118
2023-04-15 05:11:39,921 CIFAR10-Training INFO: - Ignite version: 0.4.11
2023-04-15 05:11:39,927 CIFAR10-Training INFO: - GPU Device: Tesla T4
2023-04-15 05:11:39,930 CIFAR10-Training INFO: - CUDA version: 11.8
2023-04-15 05:11:39,950 CIFAR10-Training INFO: - CUDNN version: 8700
2023-04-15 05:11:39,951 CIFAR10-Training INFO: 

2023-04-15 05:11:39,954 CIFAR10-Training INFO: Configuration:
2023-04-15 05:11:39,956 CIFAR10-Training INFO

## Important Links

1. Complete code can be found [here](https://github.com/pytorch-ignite/examples/blob/main/tutorials/intermediate/cifar10-distributed.py).
2. Example of the logs of a ClearML experiment run on this code:
   - [With torchrun](https://app.community.clear.ml/projects/14efa0ee4c114401bd06b7748314b465/experiments/83ebffd99a3f47f49dff1075252e3371/output/execution) 
   - [With default internal spawning](https://app.community.clear.ml/projects/14efa0ee4c114401bd06b7748314b465/experiments/c2b82ec98e8445f29044c94f7efc8215/output/execution)
   - [On Jupyter](https://app.community.clear.ml/projects/14efa0ee4c114401bd06b7748314b465/experiments/2fedd7447b114b36af7066cdb81fddae/output/execution)
   - [On Colab with XLA](https://app.community.clear.ml/projects/14efa0ee4c114401bd06b7748314b465/experiments/fbffb4d7f9324c57979a833a789df857/output/execution)